In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import GRU, Dense
from keras.optimizers import Adam


2024-06-17 19:21:16.943040: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-17 19:21:16.949358: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-17 19:21:17.103696: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-17 19:21:22.794902: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Load your dataset (replace 'your_dataset.csv' with the actual path to your dataset)
data = pd.read_csv('/home/rgukt/Desktop/pro/case1_considered/memory_usage.csv')

# Select relevant columns
data = data[['total_memory_mb', 'available_memory_mb', 'used_memory_mb', 'percent_memory']]

# Normalize the data
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)


In [4]:
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 100  # Length of sequences used for training
X, y = create_sequences(data_scaled, seq_length)

# Split into training and testing sets (80% train, 20% test)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


In [5]:
model = Sequential()
model.add(GRU(50, activation='relu', input_shape=(seq_length, data.shape[1])))
model.add(Dense(data.shape[1]))  # Output layer with the same number of features as input

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))


/home/rgukt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 133ms/step - loss: 0.2246 - val_loss: 0.2149
Epoch 2/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 0.1252 - val_loss: 0.1090
Epoch 3/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0469 - val_loss: 0.0361
Epoch 4/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 65ms/step - loss: 0.0157 - val_loss: 0.0093
Epoch 5/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - loss: 0.0120 - val_loss: 0.0083
Epoch 6/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0103 - val_loss: 0.0104
Epoch 7/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - loss: 0.0076 - val_loss: 0.0101
Epoch 8/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - loss: 0.0070 - val_loss: 0.0077
Epoch 9/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - loss: 0.0059 - val_loss: 0.0050
Epoch 10/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - loss: 0.0048 - val_loss: 0.0042
Epoch 11/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - loss: 0.0041 - val_loss: 0.0032
Epoch 12/50
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - loss: 0.

In [6]:
# Get the last sequence from the data
last_sequence = data_scaled[-seq_length:]

# Reshape for GRU input
last_sequence = last_sequence.reshape((1, seq_length, data.shape[1]))

# Predict the next value
next_value_scaled = model.predict(last_sequence)

# Inverse transform to get actual value
next_value_actual = scaler.inverse_transform(next_value_scaled).flatten()

print(f'Predicted next values: {next_value_actual}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step
Predicted next values: [5853.6294   1896.3871   3620.4326     67.445404]
